## Radial Basis Function (RBF) Interpolation

*Material for this worksheet can be found in the AE2220-I lecture notes Chapter 4.2.3.*

### Problem statement

Once more the goal is to find a curve passing through a set of samples (location and value). RBF interpolants are similar to the interpolants we saw in polynomial interpolation, the important difference is the choice of basis functions.  The interpolant is:
$$
\phi(\mathbf{x}) = \sum_{i=0}^N a_i \varphi( ||\mathbf{x}-\mathbf{x_i}|| )
$$
i.e. a linear sum of the $N+1$ basis functions
$$
\varphi( ||\mathbf{x}-\mathbf{x_i}|| ),\qquad i = 0,\dots,N,
$$
where $||\mathbf{x}-\mathbf{x_i}||$ represents the distance between points $\mathbf{x}$ and $\mathbf{x_i}$.  The coefficients $a_i$ are determined using the (by now) well-known interpolations conditions. 

The key difference from polynomial interpolation is that the basis functions are automatically defined in any number of dimensions $d$, as they are <i>radially symmetric</i> and <i>"centered"</i> at the grid locations $\mathbf{x}_i \in\mathbb{R}^d$.  This makes the generalization to interpolation in multiple dimensions immediate.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.labelsize': 18})
import numpy as np

Some choices of $\varphi(r)$ are (see also the notes):

In [ ]:
def phi_invquad(r,l): return 1./(1 + (l*r)**2)
def phi_invmultiquad(r,l): return 1./np.sqrt(1 + (l*r)**2)
def phi_multiquad(r,l): return np.sqrt(1 + (l*r)**2)
def phi_linear(r,l): return r
phis = [ phi_invquad, phi_invmultiquad, phi_multiquad, phi_linear]

where the $l$ parameter controls the width of these functions. For $l=1.0$ these look like...

In [ ]:
rr = np.linspace(0,2,101)
for phi in phis:
    plt.plot(rr, phi(rr,1.0), label=phi.__name__)
plt.xlabel(r'$r$'); plt.ylabel(r'$\phi$')
plt.legend()

### Radial basis function interpolation in 1d

Starting with the simplest case of 1d - setup of the problem:

In [ ]:
N_1d = 10                         ### Number of samples
a,b = -4,4                        ### Interval of interest
xi = np.linspace(a,b,N_1d+1)      ### Uniform sample locs
phi,l = phi_invquad,1.
xx = np.linspace(a,b,501)         ### Sampling of x, for plotting

def f_1d(x): return np.sin(x)  ### Target function
fi = f_1d(xi)                     ### Sample values 

In [ ]:
plt.plot(xx,f_1d(xx))
plt.plot(xi,fi,'ok')

In addition to $\varphi(x)$ we need to define a <i>metric</i>, specifying the distance between 2 points.  Typically this is the Euclidian distance, which in 1d is just:

In [ ]:
def dist_1d(x1, x2): return np.abs(x1-x2)

Together with the grid this uniquely defines the basis-functions.  Plotting them:

In [ ]:
for x in xi:
    plt.plot(xx, phi(dist_1d(x,xx),l))
plt.plot(xi,0.*xi,'ok')

Note that the basis functions depend on the grid - as did the Lagrange basis in polynomial interpolation.  The interpolation conditions can then be setup: the interpolation matrix is
$$
A_{ij} = \varphi_j(\mathbf{x}_i) = \varphi( |\mathbf{x}_i -\mathbf{x}_j | )
$$

**Exercise 1**

**(a) Define a function that builds the interpolation matrix for the RBF method. It should take 3 arguments: the sample locations *xi*, the type of basis function to use *phi* (this could be e.g. *phi_invquad*), and the scale-parameter *l*.**

In [ ]:
def interpolation_matrix_1d(xi,phi,l):
    pass   ### TODO

**(b) Plot this matrix with plt.imshow(A, interpolation='none'). What can be said about the structure of the matrix? When is $\varphi(x_j)$ large? What is the effect of the scale-parameter *l* on the matrix?**

Solve to find the cofficients $a_i$:

In [ ]:
coeffs = np.linalg.solve(A,fi)
coeffs

#### (c) Reconstruct $\phi$ from the coefficients, plot the RBF interpolant and compare it with the exact function.

In [ ]:
def reconstruct(xx):
    pass   ### TODO

**(d) What is the effect of varying the scale-parameter *l* on the quality of the reconstruction?  What happens for very large and very small values?  How would you recommend a user chooses this parameter in practice (for an unknown $f$).**

### Radial basis function interpolation in 2d

Radial basis function method is one of the primary tools for interpolating multi-dimensional data. They work well both with uniform (tensor-product) and scattered grids. The extension from 1D to 2D is very simple:

In [ ]:
N_2d = 40
N_x,N_y = 9,9
xmin,xmax,ymin,ymax = -4,4,-4,4

def f_2d(x,y): return y**2/(1.+x**2)
def dist_2d(x1,y1, x2,y2):
    return np.sqrt((x1-x2)**2+(y1-y2)**2) ### Metric

xx = np.linspace(xmin,xmax,41)  ### Sampling of x, for plotting
yy = np.linspace(ymin,ymax,41)  ### Sampling of y
xx,yy = np.meshgrid(xx, yy)     ### Tensor-product grid 

We can plot the basis function centered at $(x,y)=(0,0)$:

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
pp = phi_invquad(dist_2d(xx,yy, 0,0),l)
ax.plot_surface(xx,yy,pp,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)

**Exercise 2:**

**(a) Change the centre and scale-parameter of the RBF in the code above to better understand how RBFs look and behave.**

**(b) Selection of the sampling grid: In 2d there is more flexibility in choice of grid than in 1d.  Two options are random sampling, and sampling on a tensor-product grid.  The tensor-product grid is below - implement a randomly-sampled grid with the same number of points, and plot them.**

In [ ]:
### Tensor-product grid in 2d - uniform in each direction
xi = np.linspace(xmin,xmax,N_x)
yi = np.linspace(ymin,ymax,N_y)
xi,yi = np.meshgrid(xi,yi)
xi = xi.flatten(); yi = yi.flatten()

In [ ]:
#### TODO

In [ ]:
### Plot the grid
plt.plot(xi,yi,'ok')
plt.xlim(xmin,xmax); plt.ylim(ymin,ymax)

**(c) Plot $f(x,y)$, together with the sample locations to have an idea of what kind of data is available to the interpolation method. Change from a uniform grid to a grid with randomly distributed points.**

In [ ]:
fi = f_2d(xi,yi)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
ff = f_2d(xx,yy)
ax.plot_surface(xx,yy,ff,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)
ax.plot(xi,yi,fi,'ok')

#### Interpolation conditions

Interpolation conditions in the 2D case are identical to those in 1D. In particular $A$ is still just $A = \varphi(|\mathbf{x}_i - \mathbf{x}_j|)$.

**(d) Define a function called interpolation_matrix_2d which takes four arguments, the $x$ adn $y$ locations of the data, the RBF and the value of the shape parameter $l$. It should return the interpolation matrix $A$.**

**(e) Plot the matrix and observe its structure. Why does the matrix look like this? Is the matrix always symmetric?**

In [ ]:
#### TODO
def interpolation_matrix_2d(xi,yi,phi,l):
    pass

Solve for the coefficients $a_i$:

In [ ]:
coeffs = np.linalg.solve(A,fi)

#### Reconstruction

In [ ]:
def reconstruct_2d(xx,yy):
    out = np.zeros(xx.shape)
    for i,(x,y) in enumerate(zip(xi,yi)):
         out += coeffs[i] * phi(dist_2d(x,y,xx,yy),l)
    return out

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
zz = reconstruct_2d(xx,yy)
ax.plot_surface(xx,yy,zz,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)
ax.plot(xi, yi, fi, 'ok', label='samples')

It's difficult to compare two surfaces in 3d plots - contour plots are easier to compare:

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
cs = plt.contourf(xx,yy,ff,31)
plt.plot(xi, yi, 'ok')
plt.title('Original')
plt.subplot(122)
### Use the same levels in both plots with cs.levels
plt.contourf(xx,yy,zz,cs.levels)
plt.plot(xi, yi, 'ok')
plt.title('RBF Reconstruction')

**(f) Plot the absolute error in the form of a contour plot. Where is the largest error? Why? What would you do to reduce it?**

In [ ]:
#### TODO

### Exercise 3 (optional)

#### (a) In the list of possible RBF, the gaussian RBF was left out on purpose. Add this RBF in the code above and see what results it produces.
#### (b) Compare the results with other choices of RBFs. How does the error of this RBF behaves (make sure to use the same scale)?

The RBF method with gaussian RBFs is also called Kriging and is commonly used in many meteorological, geophysical and aerospace problems involving reconstruction of spatial data because of the possibility to obtain statistical quantities out of it. 